# CUDA tensor network contraction demo

## Requirements
- The system must have a CUDA GPU available.

In [1]:
using Tenet
using EinExprs
using Adapt
using CUDA
using BenchmarkTools

Create a random tensor network and find its contraction path:

In [2]:
# Initialize random tensor network
regularity = 6
ntensors = 10
tn = rand(TensorNetwork, ntensors, regularity)
path = einexpr(tn; optimizer=Exhaustive())

EinExpr(Symbol[], EinExpr[EinExpr([:b, :Y, :d, :M, :S, :X, :J, :V, :B], EinExpr[EinExpr([:H, :b, :Y, :c, :d, :M, :S], EinExpr[EinExpr([:a, :P, :H, :b, :E, :Z], EinExpr[EinExpr([:L, :a, :P, :H, :W], EinExpr[], Dict(:a => 3, :P => 6, :H => 6, :W => 3, :L => 8)), EinExpr([:L, :b, :E, :Z, :W], EinExpr[], Dict(:Z => 3, :b => 3, :W => 3, :L => 8, :E => 2))], Dict{Symbol, Int64}()), EinExpr([:Y, :E, :P, :a, :c, :d, :M, :Z, :S], EinExpr[EinExpr([:U, :R, :Y, :E, :D, :P], EinExpr[], Dict(:U => 3, :P => 6, :R => 6, :D => 3, :Y => 9, :E => 2)), EinExpr([:a, :R, :D, :c, :d, :M, :Z, :S, :U], EinExpr[], Dict(:a => 3, :Z => 3, :R => 6, :M => 9, :d => 3, :U => 3, :D => 3, :c => 5, :S => 3))], Dict{Symbol, Int64}())], Dict{Symbol, Int64}()), EinExpr([:X, :J, :H, :V, :c, :B], EinExpr[EinExpr([:A, :X, :J, :K, :H], EinExpr[], Dict(:A => 6, :J => 2, :H => 6, :K => 4, :X => 8)), EinExpr([:V, :c, :A, :K, :B], EinExpr[], Dict(:A => 6, :K => 4, :B => 5, :V => 8, :c => 5))], Dict{Symbol, Int64}())], Dict{Symbol,

Transform the tensors' data types to `CuArray`s:

In [3]:
cudatn = adapt(CuArray, tn)

ErrorException: CUDA driver not found

Benchmark CUDA tensor network contraction:

In [4]:
@benchmark CUDA.@sync contract(cudatn; path)

UndefVarError: UndefVarError: `cudatn` not defined

Benchmark regular tensor network contraction:

In [5]:
@benchmark CUDA.@sync contract(tn; path)

ErrorException: CUDA driver not found